In [1]:
## Load Folder Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive/')

## Move Pointer File ke DATA_UM
%cd /content/drive/MyDrive/DATA_UM

## Declare Root Project File
root_file = "/content/drive/MyDrive/DATA_UM"

Mounted at /content/drive/
/content/drive/MyDrive/DATA_UM


In [2]:
## Install necessary libraries
# &> /dev/null = Remove output Install
!pip install transformers &> /dev/null

# Import necessary libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

import re
import string

import os.path
from os import path

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
## Declare Hashtag Data
hashtag = "ultramilkxstraykids"

In [4]:
## Declare Bert Model and Load Tokenizer + Bert Untrained.

from transformers import AutoTokenizer, AutoModel

# Ganti ini kalau mau ganti model basennya.
MODEL_NAME = 'indolem/indobert-base-uncased'

# Load Token sama Base Model.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [5]:
# Build the Sentiment Classifier class
class SentimentClassifier(nn.Module):

    # Constructor class
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    # Forward propagaion class
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask,
          return_dict=False
        )
        #  Add a dropout layer
        output = self.drop(pooled_output)
        return self.out(output)

In [6]:
## Declare Class Sentiment.
class_names = ['Negative', 'Neutral', 'Positive']

## Declare Max Length Token based On Report Token.
MAX_LEN = 165

In [7]:
# Used Parameter Training
EPOCHS = 3
LR = 2e-5
BATCH_SIZE = 16

# Instantiate the model and move to classifier
model = SentimentClassifier(len(class_names))

# IF Load with GPU
model.load_state_dict(torch.load('model/model_dari_sebelah.pkl', map_location=torch.device('cpu')))

# IF Load with CPU
#model.load_state_dict(torch.load('model/'+hashtag+'/bert_model_e'+str(EPOCHS)+'_lr'+str(LR)+'_bs'+str(BATCH_SIZE)+'.pkl', map_location=torch.device('cpu')))

# Create Object Model
model = model.to(device)

In [8]:
## Load Preprocess SLANG Module

# Load Dataframe
df_tmp_nasalsabila = pd.read_csv('data/slang/colloquial-indonesian-lexicon_nasalsabila.csv')
df_slang_fendiirfan = pd.read_csv('data/slang/colloquial-indonesian-lexicon_fendiirfan.csv')
df_slang_custom = pd.read_csv('data/slang/custom_dictonary_slang.csv')
df_slang_custom_exception = pd.read_csv('data/slang/custom_dictonary_slang_dont_swap.csv')
df_slang_custom_stemming = pd.read_csv('data/slang/custom_dictonary_slang_stemming.csv')

# Transform and Delete Duplicate Data from nasalsabila
df_slang_nasalsabila = df_tmp_nasalsabila[['slang', 'formal']]
df_slang_nasalsabila.drop_duplicates(inplace=True)

# Transform and Delete Duplicate Data from fendiirfan
df_slang_fendiirfan.dropna(inplace=True)
df_slang_fendiirfan.rename(columns={'kataAlay': 'slang', 'kataBaik': 'formal'}, inplace=True)
df_slang_fendiirfan.drop_duplicates(inplace=True)

# Transform and Delete Duplicate our own data.
df_slang_custom.drop_duplicates(inplace=True)
df_slang_custom_exception.drop_duplicates(inplace=True)
df_slang_custom_stemming.drop_duplicates(inplace=True)

def remove_slang(text, df_tmp):
    # To Lowercase
    text = text.lower()

    # Pecah to array
    words = text.split(" ");

    # DF to Array
    arr_slang = df_tmp['slang'].to_numpy()
    arr_formal = df_tmp['formal'].to_numpy()
    do_not_change = df_slang_custom_exception['kata'].to_numpy()

    # Array New Text
    new_text = []

    for kata in words:
      if kata in arr_slang:
        if kata not in do_not_change:
          idx = np.where(arr_slang==kata)[0][0]
          kata = arr_formal[idx]

      new_text.append(kata)
      text_new = ' '.join(new_text)

    return text_new

<ipython-input-8-4e114e166bee>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slang_nasalsabila.drop_duplicates(inplace=True)


In [9]:
## Load Preprocess Stemming Modul
# Load and Download Library Sastrawi
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Indonesian Stopword by Sastrawi
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.5 MB/s eta 0:00:00


In [10]:
## Load Preprocess Stopword Data

# Load Data Stopword from Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# Load Data Stopword and Custom Word
stopword_satya = list(pd.read_csv('data/stopword/stopwords_satya.txt', header = None)[0])
custom_stopword = list(pd.read_csv('data/stopword/custom_stopword.txt', header = None)[0])
keep_custom_stopword = list(pd.read_csv('data/stopword/keep_custom_stopword.txt', header = None)[0])

# Declare Function / Modul Stopword
def remove_stopwords(text):
  # Removal Stopword
  text_split = text.split(' ')

  temp_text_split = []
  for i in range(len(text_split)):
    ## Remove Stopword by Satwa : gak dipake dulu ini terlalu ngilangin konteks text
    #if (text_split[i] not in stopword_satya) and (text_split[i] not in custom_stopword):
    if (text_split[i] not in custom_stopword):
      ## Keep Beberapa Kata Valence of a text
      # https://datascience.stackexchange.com/questions/15765/nlp-why-is-not-a-stop-word#:~:text=Stop%20words%20are%20usually%20thought,idf%20analysis%20for%20document%20classification).
      if (text_split[i] in keep_custom_stopword) :
        temp_text_split.append(text_split[i])
      else :
        ## Stopword Removal from data by Sastrawi
        tmp = stopword.remove(text_split[i])
        temp_text_split.append(tmp)

  final_text = ' '.join(temp_text_split)

  # will replace single word
  final_text = re.sub(r"\b[a-zA-Z]\b","",final_text)

  # will replace space more than one
  final_text = re.sub("( {2,})"," ",final_text)

  return final_text

In [11]:
def clean_text_predict(text) :

    text = str(text)

    ## TEXT TRANSFORM
    # Replace TAB with space
    text = text.replace('\t', ' ')

    # Remove username 1st
    text = re.sub('@[^\s]+','',text)

    # Remove hashtag dan @
    #''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())

    # Change "-" to space
    text = text.replace("-", " ")

    # Remove punctiation wihtout underscore
    pattern = r'[^\w\s\/\']'
    text = re.sub(pattern, ' ', text)
    text = text.replace("_", " ")
    # translator = re.compile('[%s]' % re.escape('!"#$%&\'()*+,-.:;<=>?@[\\]^_`{|}~'))
    # translator.sub(' ', text)

    # Save Special Character
    text = text.replace("/", " bimzslash ")

    # Remove angka
    #text = re.sub('[0-9]+', '', text)

    # Remove website link
    text = re.sub(r'http\S+', '', text)

    # Remove new line
    text = re.sub("\n"," ",text)

    # Transform to lowercase
    text = text.lower()

    # Slang change for spesific case (gg, dll yang menghapus multi same char)
    text = remove_slang(text ,df_slang_custom)

    # Replace perulangan huruf pada kalimat "iyaaa" to "iya"
    #text = re.sub(r'(\w)(\1{2,})', r"\1", text) #removing number and letters
    text = re.sub(r'([a-zA-Z])(\1{2,})', r"\1", text) #only remove number

    # will replace single word
    text = re.sub(r"\b[a-zA-Z]\b","",text)

    # will replace space more than one
    text = re.sub('( {2,})',' ',text)

    ## SLANG
    text = remove_slang(text ,df_slang_custom)
    text = remove_slang(text ,df_slang_nasalsabila)
    text = remove_slang(text ,df_slang_fendiirfan)

    ## STEMMING
    text = stemmer.stem(text)

    ## STOPWORD
    text = remove_stopwords(text)

    ## ROLL BACK STEMMING
    text = remove_slang(text ,df_slang_custom_stemming)

    ## TEXT TRANSFORM BUT AFTER
    # will replace single word
    # text = re.sub(r"\b[a-zA-Z]\b","",text)

    return text

In [12]:
review_text = "Keren banget kolaborasinya jadi pengen beli."
txt_clean = clean_text_predict(review_text)

encoded_review = tokenizer.encode_plus(
    txt_clean,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)
input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)

output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)

print(f'Review text\t: {review_text}')
print(f'Clean text\t: {txt_clean}')
print(f'Sentiment\t: {class_names[prediction]}')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Review text	: Keren banget kolaborasinya jadi pengen beli.
Clean text	: keren banget kolaborasi jadi pengin beli
Sentiment	: Positive


In [13]:
def predict_sentiment(text) :
  # Preprocess Data Text
  ###############################################################################
  # Proses dibawah dikomen karena data masuk data yg udh preprocessing
  text = str(text)
  # txt_clean = clean_text_predict(text)
  # Langsung Masuk Predict
  txt_clean = text
  ###############################################################################

  # Set Parameter Input
  encoded_review = tokenizer.encode_plus(
      txt_clean,
      max_length=MAX_LEN,
      add_special_tokens=True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
  )

  # Transform Data to Tensors
  input_ids = encoded_review['input_ids'].to(device)
  attention_mask = encoded_review['attention_mask'].to(device)

  # Predict
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)

  # Return Data
  return class_names[prediction]

In [14]:
df_check = pd.read_excel('data/training/ultramilk_all_data/test_new/remove_duplicate_after_preproc_for_training.xlsx', index_col=0)

In [15]:
df_check

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,...,link,text,labelling,revisi,text_final,not_balanced,balance,new_prepo,is_same_pre,jumlah
913,17-9-2022,TikTok - Komentar,ultramilk,dontstopyourmove,renata👊,rinn_lovers,https://www.tiktok.com/@rinn_lovers,0.0,NaN,NaN,...,https://www.tiktok.com/@uglyeoja/video/7143900...,Yuviii,1,1,yuvi,Neutral,Neutral,yuvi,True,1
2602,2022-07-09 00:00:00,Instagram,ultramilk,#DontStopYourMove,حنفة اليس فترىانى,_hanifals_,Nan,NaN,NaN,NaN,...,https://www.instagram.com/p/CftPm-yho6M,@__inayatulll__ yuniiiii😍😂,1,1,yuni,Neutral,Neutral,yuni,True,1
2591,2022-07-08 00:00:00,Instagram,ultramilk,#DontStopYourMove,Zia,zzizabae,Nan,NaN,NaN,NaN,...,https://www.instagram.com/p/Cfu2lbTh2y8,Yuna 😍,1,1,yuna,Neutral,Neutral,yuna,True,1
53,2020-01-01,Mentionlytics,Mentionlytics,Mentionlytics,Mentionlytics,Mentionlytics,Mentionlytics,0.0,0.0,0.0,...,Mentionlytics,"Halo teman - teman, saya mau sharing nih, peng...",2,2,halo teman teman mau sharing alam juang kumpul...,Negative,Positive,halo teman teman mau sharing alam juang kumpul...,False,98
91,2020-01-01,Mentionlytics,Mentionlytics,Mentionlytics,Mentionlytics,Mentionlytics,Mentionlytics,0.0,0.0,0.0,...,Mentionlytics,sebulan penuh bersama ultramilk jaya jaya jaya...,2,1,bulan penuh sama ultramilk jaya jaya jaya haha...,Negative,Neutral,bulan penuh sama ultramilk jaya jaya jaya tert...,False,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14012,2023-11-30 00:00:00,Instagram,ultramilk,#iamULTRAoddinary,chik mia,mia.chik,Nan,NaN,NaN,NaN,...,https://www.instagram.com/p/CzFsyLArKoe,kesyukaanku,2,2,kesyukaanku,Neutral,Positive,kesyukaanku,True,1
1123,2023-01-23 00:00:00,Instagram,Manipulation,Manipulation,Leoni Margaretha,leoni_margaretha,Nan,NaN,NaN,NaN,...,https://www.instagram.com/p/Cnoxvk1SgCh,@terusraun yess bener kak... tp bukankah harus...,0,0,yess benar kakak bukankah tim cek ricek dulu c...,Neutral,Negative,yess benar kakak tapi bukankah harus tim cek r...,False,21
64,2023-11-29,Twitter,ultramilk,iamultraoddinary,COPPAMAGZ,@coppamagz,769863624982601728,0.0,2.0,0.0,...,https://twitter.com/coppamagz/status/172972752...,Hayoo mana nih yang kemarin pas event #iamULTR...,1,4,hayoo mana kemarin pas event iamultraoddinary ...,Neutral,Neutral,hayoo mana kemarin pas event iamultraoddinary ...,False,118
49,2022-05-05 00:00:00,Instagram,Manipulation,Manipulation,Jaguar,jaguarfarizal,Nan,NaN,NaN,NaN,...,https://www.instagram.com/p/CbBnsdpJ1MV,Jangan ada yang ikutan deh sama program receh...,0,0,jangan ikut sama program tidak tarik ultramilk...,Negative,Negative,jangan ada ikut sama program tidak tarik dari ...,False,133


In [16]:
df_check['model_baru'] = df_check['new_prepo'].apply(lambda x: predict_sentiment(x))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
df_check.to_excel('data/training/ultramilk_all_data/aaa_check_predicted_nobalance_balance.xlsx')